In [6]:
#A função desse código é automatizar a obtenção de dados das ações brasileiras e aplicar métricas baseadas nos dividendos 
#pagos pelas empresas, a matemática financeira utilizada é inspirada em Décio Bazin, que consiste em determinar um preço teto 
#a se pagar pela ação com base nos proventos distribuidos nos últimos anos.

#ATENÇÃO! ESSA FERRAMENTA NÃO É  UMA INDICAÇÃO DE COMPRA, O PROPÓSITO DO CÓDIGO É EDUCACIONAL E ANALÍTICO, 
#AS DECISÕES DE INVESTIMENTOS SÃO 100% RESPONSABILIDADE DO USUÁRIO.


#---------------------------------------#
#Caso não possua as bibliotecas execute:
#!pip install pandas
#!pip install numpy
#!pip install yfinance
#!pip install datetime
#---------------------------------------#

#Importa as bibliotecas
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from datetime import datetime, timedelta

#-----------------------------------------#

#Dividend Yield alvo %
divd_target = 0.06

#-----------------------------------------#

#----------------------------------------------------------------------------------------------------------------------------------------------------#

#Importa os dados das ações brasileiras, ativos selecionados manualmente com base em seus setores de atuação.
dt_ativos = ['QUAL3', 'BRAP4', 'BRAP3', 'AESB3', 'VBBR3', 'WIZS3', 'BRSR6', 'TRPL4', 'BRSR3', 'VIVT3', 'CGAS5', 'CSMG3', 'TAEE11', 'TAEE3', 'TAEE4', 'BRKM3', 'BRKM5', 'BBSE3', 'BRIV4', 'CGAS3', 'TRPL3', 'SANB4', 'SANB3', 'SANB11', 'ITSA4', 'ITSA3', 'SAPR4', 'BRSR5', 'SAPR3', 'GRND3', 'ENAT3', 'CPLE6', 'VALE3', 'CPLE3', 'EGIE3', 'PSSA3', 'ITUB3', 'FLRY3', 'UGPA3', 'BBDC3', 'ABCB4', 'ODPV3', 'SAPR11', 'BBAS3', 'LEVE3', 'CCRO3', 'BBDC4', 'ROMI3', 'ITUB4', 'CMIG4', 'MYPK3', 'BMEB4', 'UNIP3', 'ABEV3', 'CLSC3', 'UNIP5', 'UNIP6', 'CLSC4', 'CPFE3', 'ENBR3', 'ENGI4', 'SULA11', 'B3SA3', 'SULA4', 'SULA3', 'NEOE3', 'KLBN11', 'KLBN4', 'TUPY3', 'KLBN3', 'ELET5', 'ALUP3', 'ALUP4', 'ALUP11', 'ELET6', 'CSAN3', 'CMIG3', 'HYPE3', 'FESA4', 'SLCE3', 'BMEB3', 'CPLE5', 'FESA3', 'ENGI3', 'CPLE11', 'ELET3', 'BRKM6', 'BPAC5', 'SBSP3', 'TIMS3', 'EQTL3', 'BPAC11', 'BPAC3', 'BRIV3', 'ENGI11', 'AURE3', 'MEGA3']

ativos = []
for a in dt_ativos:
    if len(a) <= 7:
        ativos.append(a + '.SA')
        
divd_acoes = yf.download(ativos, start = '2017-01-01', end = '2022-12-31', actions=True)['Dividends']


divdatt_acoes = yf.download(ativos.copy(), start = datetime.today() - timedelta(days = 365), end = datetime.today(), actions=True)['Dividends']


close_acoes = yf.download(ativos, start = datetime.today() - timedelta(3), end = datetime.today(), actions=True)['Close']

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#

#Tratamento de Dados, aplicação de métricas e cálculos.
df = pd.melt(divd_acoes.reset_index(), id_vars=['Date'], var_name = 'Ticker', value_name = 'Dividendos').sort_values(['Ticker','Dividendos','Date'], ascending = True).fillna(0)
df['Date'] = pd.to_datetime(df['Date'])
df['Ano'] = df['Date'].dt.isocalendar()['year']

dfatt = pd.melt(divdatt_acoes.reset_index(), id_vars=['Date'], var_name = 'Ticker', value_name = 'Dividends').sort_values(['Ticker','Dividends','Date'], ascending = True).fillna(0)
dfatt['Date'] = pd.to_datetime(dfatt['Date'])
dfatt['Ano'] = dfatt['Date'].dt.isocalendar()['year']

calcdiv = df.groupby(['Ticker']).agg({'Dividendos' : 'sum'}).sort_values(['Dividendos',], ascending = False)
calcdiv12 = dfatt.groupby(['Ticker']).agg({'Dividends' : 'sum'}).sort_values(['Dividends',], ascending = False)

soma_dividendos = calcdiv.loc[calcdiv.Dividendos !=0, :]
x = soma_dividendos.reset_index()
soma_dividendos12 = calcdiv12.loc[calcdiv12.Dividends !=0, :]
xx = soma_dividendos12.reset_index()

df1 = pd.melt(close_acoes.reset_index(), id_vars=['Date'], var_name = 'Ticker', value_name = 'Preço Atual').sort_values(['Ticker','Preço Atual','Date'], ascending = True).fillna(0)

y = df1[df1.Date == df1.Date.max()]

z = x.merge(y, how='inner', on='Ticker')

z['DY Médio 6Y'] = z['Dividendos'] / 6
z['Preço Teto'] = z['DY Médio 6Y'] / divd_target
calc = z['calc'] = z['Preço Teto'] / z['Preço Atual'] - 1
z['Margem_Segurança'] = calc.copy() *100
z['DY_Médio'] = z['DY Médio 6Y'] / z['Preço Atual'] * 100

pd.set_option('display.max_rows', 500)
z.replace([np.inf, -np.inf], np.nan, inplace=True)
z.dropna(inplace = True)
z.sort_values(['Margem_Segurança'], ascending = False)
z['gp'] = z['Dividendos'] *2
z.rename(columns={'Date': 'Data'}, inplace = True)
dtt = z.groupby(['Ticker', 'Data', 'Preço Atual','Preço Teto', 'Margem_Segurança','DY_Médio']).agg({'gp' : 'mean'}).sort_values(['DY_Médio'], ascending = False)
dtt1 = dtt.drop(columns = ['gp'])
test = dtt1.reset_index()
pd.options.display.float_format = '{:.2f}'.format
test1 = test.loc[test.DY_Médio >= divd_target *100]
DataFrame = test1.merge(calcdiv12, 'inner', on = 'Ticker')
DataFrame['DY% Atual'] = DataFrame['Dividends'] / DataFrame['Preço Atual'] *100
DataFrame = DataFrame.drop(columns = ['Dividends'])

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------#
#Resultado Final
DataFrame

[*********************100%***********************]  97 of 97 completed
[*********************100%***********************]  97 of 97 completed
[*********************100%***********************]  97 of 97 completed

3 Failed downloads:
- SULA11.SA: No data found for this date range, symbol may be delisted
- SULA3.SA: No data found for this date range, symbol may be delisted
- SULA4.SA: No data found for this date range, symbol may be delisted


,Ticker,Data,Preço Atual,Preço Teto,Margem_Segurança,DY_Médio,DY% Atual
0,QUAL3.SA,2023-01-12,6.54,16.72,155.65,15.34,3.91
1,BRKM5.SA,2023-01-12,24.13,50.75,110.30,12.62,14.06
2,BRKM3.SA,2023-01-12,24.50,50.75,107.13,12.43,13.85
3,BRAP3.SA,2023-01-12,27.97,48.68,74.04,10.44,10.63
4,BRSR6.SA,2023-01-12,9.65,16.78,73.84,10.43,9.13
5,WIZS3.SA,2023-01-12,6.94,11.66,68.00,10.08,8.47
6,BRAP4.SA,2023-01-12,32.01,53.55,67.28,10.04,10.22
7,ENAT3.SA,2023-01-12,14.62,23.74,62.41,9.74,11.69
8,BRSR3.SA,2023-01-12,10.79,16.78,55.47,9.33,8.17
9,TRPL4.SA,2023-01-12,22.80,33.21,45.64,8.74,4.66
